# Entrenamiento y predicción con BigQuery ML

In [17]:
%%bigquery
SELECT
    *
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:02<00:00,  2.37rows/s]


,age,workclass,functional_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,27,Private,29261,HS-grad,9,Married-AF-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>50K
1,47,Self-emp-not-inc,117865,HS-grad,9,Married-AF-spouse,Craft-repair,Husband,White,Male,0,0,90,United-States,<=50K
2,29,Private,95423,HS-grad,9,Married-AF-spouse,Transport-moving,Husband,White,Male,0,0,80,United-States,<=50K
3,26,Private,98155,HS-grad,9,Married-AF-spouse,Sales,Husband,White,Male,0,0,55,United-States,<=50K
4,30,Private,398827,HS-grad,9,Married-AF-spouse,Adm-clerical,Husband,White,Male,0,0,60,United-States,<=50K


In [ ]:
#! pip3 install -U  -q --user tensorflow==2.5 \
#    tensorflow-data-validation==1.2 \
#    tensorflow-transform==1.2 \
#    tensorflow-io==0.18 


## Introducción

Este ejemplo muestra cómo entrenar y evaluar un modelo de clasificación binaria con BQML. También usaremos este modelo para generar predicciones.

Las dos primeras partes del tutorial explican cómo extraer los datos y prepararlos antes de entrenar su modelo.

La última parte del tutorial profundiza en el código de entrenamiento que se usa para este modelo, con un enfoque particular en los requisitos para hacerlo compatible con BigQuery ML.

## Conjunto de Datos

Este tutorial utiliza el conjunto de datos de ingresos del censo de los Estados Unidos proporcionado por el repositorio de aprendizaje automático de UC Irvine. Este conjunto de datos contiene información sobre personas de una base de datos del Censo de 1994, incluida la edad, la educación, el estado civil, la ocupación y si ganan más de $50,000 al año.

El conjunto de datos ahora existe en los conjuntos de datos públicos de BigQuery:

`bigquery-public-data.ml_datasets.census_adult_income`

## El objetivo del modelo

El objetivo es entrenar un modelo de clasificación binaria con BigQuery ML que prediga si una persona gana más de $50,000 al año (etiqueta de destino) en función de otra información del censo sobre la persona (características).

## Configuración del proyecto

In [5]:
# Data processing
import pandas as pd

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# BigQuery API
from google.cloud import bigquery 

In [6]:
PROJECT = 'bq-experiments-350102'

In [7]:
import os
os.environ['PROJECT'] = PROJECT

## Flujo de trabajo con un modelo de BigQuery

In this section we will build a BQML model from scratch. We will perform the following steps:
- Preparación de datos
- Creación de modelo
- Entrenamiento modelo
- Evaluación del modelo
- Sirviendo el modelo

### Importaciones

In [15]:
# Procesamiento de datos
import pandas as pd

# Visualizaciones
import matplotlib.pyplot as plt
import seaborn as sns

# BigQuery API
from google.cloud import bigquery 

# TensorFlow data validation
import tensorflow_data_validation as tfdv

# Mostrar versiones de software
print(__import__('sys').version)
print(pd.__version__)

2022-12-06 16:37:18.802572: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
1.3.5


## Preparación de datos

### Procesar columnas numéricas y categóricas
Los conjuntos de datos del censo contienen números y cadenas que necesitamos para convertir los datos de cadena en números para poder entrenar el modelo.

### BigQuery ML admite el manejo de datos categóricos:

Para todas las columnas no numéricas que no sean TIMESTAMP, BigQuery ML realiza una transformación de codificación one-hot. Esta transformación genera una característica separada para cada valor único en la columna.

### Crear cliente de BigQuery
Cree un cliente de BigQuery para agrupar la configuración necesaria para las solicitudes de API.

In [8]:
client = bigquery.Client(location='US', project=PROJECT)

### Crear conjunto de datos de BigQuery
Crearemos un conjunto de datos llamado censo_tutorial, dentro de este conjunto de datos crearemos nuestro modelo BQML. **NOTA:** solo ejecute este código una vez.

In [ ]:
# Dataset and table information
dataset_name = 'census_example'

# Create BigQuery dataset
dataset = client.create_dataset(dataset_name)

## Extraiga conjuntos de datos de entrenamiento y evaluación:
En este caso, dividiremos nuestros datos en 80/10/10 para entrenamiento, validación y prueba.

Para eML, desea un muestreo repetible de los datos que tiene en BigQuery. Para obtener los datos de validación: cambie el < 8 en la consulta anterior a = 8, y para los datos de prueba, cámbielo a = 9. De esta manera, obtiene el 10 % de las muestras en validación y el 10 % en prueba.

#### Todos los datos

In [9]:
query = """
SELECT
    age,
    workclass,
    functional_weight,
    education,
    education_num,
    marital_status,
    occupation,
    relationship,
    race,
    sex,
    capital_gain,
    capital_loss,
    hours_per_week,
    native_country,
    income_bracket
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`
"""
dataset = client.query(query).to_dataframe()

### Datos de entrenamiento

[Muestreo repetible](https://towardsdatascience.com/ml-design-pattern-5-repeatable-sampling-c0ccb2889f39)

In [10]:
query = """
SELECT
    age,
    workclass,
    functional_weight,
    education,
    education_num,
    marital_status,
    occupation,
    relationship,
    race,
    sex,
    capital_gain,
    capital_loss,
    hours_per_week,
    native_country,
    income_bracket
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`
WHERE
  MOD(ABS(FARM_FINGERPRINT(CAST(functional_weight AS STRING))), 10) < 8
"""
train_dataset = client.query(query).to_dataframe()

#### Datos de evaluación

In [11]:
query = """
SELECT
    age,
    workclass,
    functional_weight,
    education,
    education_num,
    marital_status,
    occupation,
    relationship,
    race,
    sex,
    capital_gain,
    capital_loss,
    hours_per_week,
    native_country,
    income_bracket
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`
WHERE
  MOD(ABS(FARM_FINGERPRINT(CAST(functional_weight AS STRING))), 10) = 8
"""
eval_dataset = client.query(query).to_dataframe()

#### Datos de prueba

In [12]:
query = """
SELECT
    age,
    workclass,
    functional_weight,
    education,
    education_num,
    marital_status,
    occupation,
    relationship,
    race,
    sex,
    capital_gain,
    capital_loss,
    hours_per_week,
    native_country,
    income_bracket
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`
WHERE
  MOD(ABS(FARM_FINGERPRINT(CAST(functional_weight AS STRING))), 10) = 9
"""
test_dataset = client.query(query).to_dataframe()

#### Longitud del dataframe

Indique la longitud de los datos de entrenamiento y prueba. Esto validará que el conjunto de datos contenga datos antes de que comencemos a procesarlo.

In [13]:
len(dataset), len(train_dataset), len(eval_dataset), len(test_dataset)

(32561, 26192, 3268, 3101)

## Exploración de datos

In [16]:
stats = tfdv.generate_statistics_from_dataframe(
    dataframe=dataset,
    stats_options=tfdv.StatsOptions(
        label_feature="income_bracket", sample_rate=1, num_top_values=50
    ),
)

In [ ]:
tfdv.visualize_statistics(stats)

## Visualiza los datos

Pandas proporciona el método .corr. Se utiliza para encontrar la correlación por pares de todas las columnas en el marco de datos. Cualquier valor de na se excluye automáticamente. Para cualquier columna de tipo de datos no numérico en el marco de datos, se ignora.

El término correlación se refiere a una relación mutua o asociación entre cantidades.

Cuanto más cerca esté ρ de 1, más se asocia un aumento en una variable con un aumento en la otra. Por otro lado, cuanto más cerca esté ρ de -1, el aumento en una variable resultará en una disminución en la otra. Tenga en cuenta que si X e Y son independientes, entonces ρ está cerca de 0, ¡pero no al revés! En otras palabras, la correlación de Pearson puede ser pequeña incluso si existe una fuerte relación entre dos variables.

In [ ]:
hmap = dataset.corr(method='pearson')
plt.subplots(figsize=(12, 9))
sns.heatmap(hmap, vmax=0.8 ,annot=True, cmap="BrBG", square=True)

Cuando observamos las características numéricas, no tienen una fuerte correlación. Los atributos numéricos tienen un número significativo de valores únicos:

functional_weight, tiene más de 28,000 valores únicos para un conjunto de ~32,000 valores.
Esto puede indicar que esta característica podría no ser un predictor significativo.

#### Visualizar interacciones

In [ ]:
sns.pairplot(dataset, hue='income_bracket')

En este gráfico puede ver las diferentes muestras y sus valores, esto es importante cuando está haciendo ingeniería de características.

### Recuento de >50K y <=50K

In [ ]:
sns.displot(dataset, x='income_bracket')

Hay un desequilibrio en la proporción de etiquetas, con 24720 valores para <=50K y 7841 para => 50K. sin embargo, el 24% del total debería ser suficiente para determinar la clase a través de patrones. En otros casos se pueden aplicar técnicas como boosting.

## Ingeniería de características
#### Análisis de datos numéricos
Haremos un análisis de datos numéricos para verificar las diferentes características y la correlación con el nivel de ingresos.

En las variables categóricas, hay tres atributos con valores desconocidos/faltantes:

workclass (6%)
occupation (6%)
native_country (2%)

In [ ]:
# Comprobación de campos vacíos (NULO) O (?) y su porcentaje general
query = """
SELECT
  COUNTIF(workclass IS NULL 
    OR LTRIM(workclass) LIKE '?') AS workclass,
  ROUND(COUNTIF(workclass IS NULL 
    OR LTRIM(workclass) LIKE '?') / COUNT(workclass) * 100) 
    AS workclass_percentage,
  COUNTIF(occupation IS NULL 
    OR LTRIM(occupation) LIKE '?') AS occupation,  
  ROUND(COUNTIF(occupation IS NULL 
    OR LTRIM(occupation) LIKE '?') / COUNT(occupation) * 100) 
    AS occupation_percentage,
  COUNTIF(native_country IS NULL 
    OR LTRIM(native_country) LIKE '?') AS native_country,
  ROUND(COUNTIF(native_country IS NULL 
    OR LTRIM(native_country) LIKE '?') / COUNT(native_country) * 100) 
    AS native_country_percentage
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`
"""
client.query(query).to_dataframe()

In [ ]:
# Comprobación de los valores de workclass.
query = """
SELECT
  workclass,
  COUNT(workclass) AS total_workclass
FROM 
  `bigquery-public-data.ml_datasets.census_adult_income`
GROUP BY workclass
ORDER BY total_workclass DESC
"""
client.query(query).to_dataframe()

In [ ]:
# Comprobación de los valores de occupation.
query = """
SELECT
  occupation,
  COUNT(occupation) AS total_occupation
FROM 
  `bigquery-public-data.ml_datasets.census_adult_income`
GROUP BY occupation
ORDER BY total_occupation DESC
"""
client.query(query).to_dataframe()

In [ ]:
# Comprobación de los valores de native_country.
query = """
SELECT
  native_country,
  COUNT(native_country) AS total_native_country
FROM 
  `bigquery-public-data.ml_datasets.census_adult_income`
GROUP BY native_country
ORDER BY total_native_country DESC
"""
client.query(query).to_dataframe()

Echemos un vistazo a los siguientes atributos numéricos:

- **capital_gain** y **capital_loss** tienen cada uno cerca de 100 valores únicos, aunque la mayoría de sus instancias tienen valores cero.

- **capital_gain** tiene 72 % de instancias con valores cero para menos de 50 000 y 19 % de instancias con valores cero para > 50 000.

- **capital_loss** tiene 73 % de instancias con valores cero para menos de 50 000 y 21 % de instancias con valores cero para > 50 000.

- Esto implica que **capital_gain** o **capital_loss** tampoco serán predictores significativos.

In [ ]:
# Evaluación de la educación
query = """
SELECT
  education,
  education_num      
FROM 
  `bigquery-public-data.ml_datasets.census_adult_income`
GROUP BY education, education_num
ORDER BY education_num
"""
client.query(query).to_dataframe()

**educación** y **educación_número** son indicadores del mismo atributo y están completamente correlacionados con el mapeo directo, tiene sentido eliminar uno de ellos durante la selección de características.

In [ ]:
# Explore Núm. de Educación vs. Ingresos
g = sns.catplot(x="education_num", y="income_bracket", data=dataset,kind="bar", 
                height = 6,palette = "muted")
g.despine(left=True)
g = g.set_ylabels(">50K probability")

In [ ]:
# Explorando marital_status
query = """
SELECT
  marital_status            
FROM 
  `bigquery-public-data.ml_datasets.census_adult_income`
GROUP BY marital_status
"""
client.query(query).to_dataframe()

In [ ]:
# Explorando relationship
query = """
SELECT
  relationship            
FROM 
  `bigquery-public-data.ml_datasets.census_adult_income`
GROUP BY relationship
"""
client.query(query).to_dataframe()

Dado que la **relationship** y el **marital_status** son funciones que describen un estado similar, es posible que podamos eliminar la **relationship** y mantener **marital_status** y crear una nueva función con nuevos valores.

## Model Training

Comenzaremos el entrenamiento del modelo utilizando el conjunto de datos público:

La declaración **CREATE MODEL** se usa para crear y entrenar el modelo llamado censo_tutorial.census_modelo.

**CREATE OR REPLACE MODEL**

Crea y entrena un modelo y reemplaza un modelo existente con el mismo nombre en el conjunto de datos especificado.

**OPTIONS(model_type='logistic_reg', input_label_cols=['income_bracket'])**

La declaración indica que está creando un modelo de regresión logística. Esta opción crea un modelo de regresión logística o un modelo de regresión logística multiclase. Para los modelos de regresión logística, la columna de la etiqueta debe contener solo dos valores distintos. Cuando cree un modelo de regresión logística multiclase, especifique datos de entrenamiento que contengan más de dos etiquetas únicas.

- **model_type:** logistic_reg crea un modelo de regresión logística o un modelo de regresión logística multiclase.

- **auto_class_weights:** de forma predeterminada, los datos de entrenamiento utilizados para crear un modelo de regresión logística multiclase no están ponderados. Si las etiquetas en los datos de entrenamiento están desequilibradas, el modelo puede aprender a predecir la clase de etiquetas más popular en mayor medida, lo que puede no ser deseable. Los pesos de clase se pueden usar para equilibrar las etiquetas de clase y se pueden usar para regresiones logísticas y logísticas multiclase. Si se establece en verdadero, los pesos de cada clase se calculan en proporción inversa a la frecuencia de esa clase. Para equilibrar cada clase, use la siguiente fórmula: TOTAL_INPUT_ROWS / (INPUT_ROWS_FOR_CLASS_N * NUMBER_OF_UNIQUE_CLASSES)

- **data_split_method:** el método para dividir los datos de entrada en conjuntos de entrenamiento y evaluación. Los datos de entrenamiento se utilizan para entrenar el modelo. Los datos de evaluación se utilizan para evitar el sobreajuste a través de una parada anticipada. El valor predeterminado es auto_split.

- **input_label_cols:** los nombres de las columnas de etiquetas en los datos de entrenamiento. input_label_cols acepta una matriz de cadenas, pero solo se admite un elemento de matriz para los modelos linear_reg y logistic_reg. Si no se especifica input_label_cols, se utiliza la columna denominada "etiqueta" en los datos de entrenamiento. Si ninguno existe, la consulta falla.

- **max_iterations:** el número máximo de iteraciones de entrenamiento (pasos). El valor predeterminado es 20.

Cuando utiliza una declaración CREATE MODEL, el tamaño del modelo debe ser de 90 MB o menos o la consulta falla. Por lo general, si todas las variables categóricas son cadenas cortas, se admite una cardinalidad total de características (dimensión del modelo) de 5 a 10 millones. La dimensionalidad depende de la cardinalidad y la longitud de las variables de cadena.

Cuando utiliza una declaración CREATE MODEL, la columna de la etiqueta no puede contener valores NULL. Si la columna de la etiqueta contiene valores NULL, la consulta falla.

### Estandarice los datos para la convergencia de modelos

1. Para todas las columnas numéricas, BigQuery ML estandariza y centra la columna en cero antes de pasarla al entrenamiento.

2. En la consulta SQL omitimos las siguientes columnas para datos de entrenamiento:

['functional_weight', 'education', 'sex', 'relationship']

3. BQML convertirá las características categóricas a numéricas.

4. Los valores desconocidos en **workclass** y **native_country** se reemplazan con Private y United States respectivamente.

Se eliminan las instancias con valores desconocidos para la **occupation*.

5. Elimina la **relationship** y usa **marital_status** de manera simplificada. (Puede usar Chi Square, que se usa comúnmente para probar relaciones entre variables categóricas (martial_status vs. relationship).

6. Se eliminan los duplicados en el conjunto de trenes.

El resultado de la creación del modelo será un marco de datos vacío, esto es normal.

In [ ]:
# Entrenar un BQML model
train_query = """
CREATE OR REPLACE MODEL `census_example.census_model`
  OPTIONS (
      model_type='logistic_reg',
      auto_class_weights=true,
      data_split_method='no_split',
      input_label_cols=['income_bracket'],
      max_iterations=15) AS
  SELECT
      age,
      CASE 
        WHEN workclass IS NULL THEN 'Private' 
        WHEN LTRIM(workclass) LIKE '?' THEN 'Private'
        ELSE workclass
      END AS workclass,
      CASE 
        WHEN native_country IS NULL THEN 'United States' 
        WHEN LTRIM(native_country) LIKE '?' THEN 'United States'
        ELSE native_country
      END AS native_country,        
      CASE 
        WHEN LTRIM(marital_status) IN 
          (
           'Never-married',
           'Divorced',
           'Separated',
           'Widowed'
          ) THEN 'Single' 
        WHEN LTRIM(marital_status) IN 
          (
           'Married-civ-spouse',
           'Married-spouse-absent',
           'Married-AF-spouse'
          ) THEN 'Married' 
        ELSE NULL 
      END AS marital_status,
      education_num,
      occupation,
      race,       
      hours_per_week,        
      income_bracket
    FROM   
      `bigquery-public-data.ml_datasets.census_adult_income`
    WHERE
      MOD(ABS(FARM_FINGERPRINT(CAST(functional_weight AS STRING))), 10) < 8
      AND (occupation IS NOT NULL OR LTRIM(occupation) NOT LIKE '?%')
    GROUP BY  1, 2, 3, 4, 5, 6, 7, 8, 9
"""
client.query(train_query)

### Información del modelo
Un algoritmo de ML construye un modelo examinando muchos ejemplos e intentando encontrar un modelo que minimice la pérdida. Este proceso se denomina minimización empírica del riesgo.

La pérdida es la penalización por una mala predicción, un número que indica qué tan mala fue la predicción del modelo en un solo ejemplo. Si la predicción del modelo es perfecta, la pérdida es cero; de lo contrario, la pérdida es mayor. El objetivo de entrenar un modelo es encontrar un conjunto de ponderaciones y sesgos que tengan una pérdida baja, en promedio, en todos los ejemplos.

In [ ]:
training_info = """
SELECT
  training_run,
  iteration,
  loss,
  duration_ms,
  learning_rate
FROM
  ML.TRAINING_INFO(MODEL `census_example.census_model`)
ORDER BY iteration ASC
"""
client.query(training_info).to_dataframe()

### Evaluación del modelo

In [ ]:
# Realizar la evaluación del modelo
query_evaluate = """
SELECT 
  precision,
  recall,
  accuracy,
  f1_score,
  log_loss,
  roc_auc
FROM ML.EVALUATE (MODEL `census_example.census_model`, 
  (
    SELECT
      age,
      CASE 
        WHEN workclass IS NULL THEN 'Private' 
        WHEN LTRIM(workclass) LIKE '?' THEN 'Private'
        ELSE workclass
      END AS workclass,
      CASE 
        WHEN native_country IS NULL THEN 'United States' 
        WHEN LTRIM(native_country) LIKE '?' THEN 'United States'
        ELSE native_country
      END AS native_country,        
      CASE 
        WHEN LTRIM(marital_status) IN 
          (
            'Never-married',
            'Divorced',
            'Separated',
            'Widowed'
            ) THEN 'Single' 
        WHEN LTRIM(marital_status) IN 
          (
            'Married-civ-spouse',
            'Married-spouse-absent',
            'Married-AF-spouse'
            ) THEN 'Married' 
        ELSE NULL 
      END AS marital_status,
      education_num,
      occupation,
      race,       
      hours_per_week,        
      income_bracket
    FROM   
      `bigquery-public-data.ml_datasets.census_adult_income`
    WHERE
      MOD(ABS(FARM_FINGERPRINT(CAST(functional_weight AS STRING))), 10) = 8
      AND (occupation IS NOT NULL OR LTRIM(occupation) NOT LIKE '?%')
    GROUP BY 1, 2, 3, 4, 5, 6, 7, 8, 9
    ))
"""
evaluation_job = client.query(query_evaluate).to_dataframe()

### Resultados del modelo
Al examinar la precisión de la ROC (roc_auc) impresa durante la evaluación, debería ver que su modelo finalmente aprendió a predecir el nivel de ingresos de una persona con una precisión de alrededor del 86 %. Puede mejorarlo?

In [ ]:
evaluation_job

### ROC
La función de salida ML.ROC_CURVE incluye varias filas con métricas para diferentes valores de umbral para el modelo. Las métricas incluyen:

- threshold
- recall
- false_positive_rate
- true_positives
- false_positives
- true_negatives
- false_negatives

In [ ]:
# Evaluación del Modelo
query_roc_curve = """
SELECT
  threshold,
  recall,
  false_positive_rate,
  true_positives,
  false_positives,
  true_negatives,
  false_negatives
FROM
  ML.ROC_CURVE(MODEL `census_example.census_model`,
  (
    SELECT
      age,
      CASE 
        WHEN workclass IS NULL THEN 'Private' 
        WHEN LTRIM(workclass) LIKE '?' THEN 'Private'
        ELSE workclass
      END AS workclass,
      CASE 
        WHEN native_country IS NULL THEN 'United States' 
        WHEN LTRIM(native_country) LIKE '?' THEN 'United States'
        ELSE native_country
      END AS native_country,        
      CASE 
        WHEN LTRIM(marital_status) IN 
          (
            'Never-married',
            'Divorced',
            'Separated',
            'Widowed'
            ) THEN 'Single' 
        WHEN LTRIM(marital_status) IN 
          (
            'Married-civ-spouse',
            'Married-spouse-absent',
            'Married-AF-spouse'
            ) THEN 'Married' 
        ELSE NULL 
      END AS marital_status,
      education_num,
      occupation,
      race,       
      hours_per_week,        
      income_bracket
    FROM   
       `bigquery-public-data.ml_datasets.census_adult_income`
    WHERE
      MOD(ABS(FARM_FINGERPRINT(CAST(functional_weight AS STRING))), 10) = 8
      AND (occupation IS NOT NULL OR LTRIM(occupation) NOT LIKE '?%')
    GROUP BY  1, 2, 3, 4, 5, 6, 7, 8, 9))
"""
roc_curve_job = client.query(query_roc_curve).to_dataframe()
roc_curve_job

### Visualizar el historial del modelo

In [ ]:
# Visualice la historia para la precisión.
plt.title('BigQuery ML Model accuracy')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.02])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
fpr = roc_curve_job['false_positive_rate']
tpr = roc_curve_job['recall']
plt.plot(fpr, tpr, color='darkorange')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.show()

## Modelo de predicciones
### Generar predicciones
Ahora usará su modelo para predecir resultados. La siguiente consulta usa ML.PREDICT. La consulta devuelve estas columnas:
- income_bracket
- predicted_income_bracket
- predicted_income_bracket_probs
- All other columns specified in query.

**Nota:** normalmente para la predicción, su etiqueta estará vacía, en este ejemplo la usamos para poder comparar el resultado del modelo con la etiqueta esperada.

In [ ]:
query_prediction = """
SELECT
  income_bracket,
  predicted_income_bracket,
  predicted_income_bracket_probs
FROM
  ML.PREDICT(MODEL `census_example.census_model`,
  (
    SELECT
      age,
      CASE 
        WHEN workclass IS NULL THEN 'Private' 
        WHEN LTRIM(workclass) LIKE '?' THEN 'Private'
        ELSE workclass
      END AS workclass,
      CASE 
        WHEN native_country IS NULL THEN 'United States' 
        WHEN LTRIM(native_country) LIKE '?' THEN 'United States'
        ELSE native_country
      END AS native_country,        
      CASE 
        WHEN LTRIM(marital_status) IN 
        (
            'Never-married',
            'Divorced',
            'Separated',
            'Widowed'
            ) THEN 'Single' 
        WHEN LTRIM(marital_status) IN 
          (
            'Married-civ-spouse',
            'Married-spouse-absent',
            'Married-AF-spouse'
            ) THEN 'Married' 
        ELSE NULL 
      END AS marital_status,
      education_num,
      occupation,
      race,       
      hours_per_week,        
      income_bracket
    FROM   
       `bigquery-public-data.ml_datasets.census_adult_income`
    WHERE
      MOD(ABS(FARM_FINGERPRINT(CAST(functional_weight AS STRING))), 10) = 9
      AND occupation IS NOT NULL AND LTRIM(occupation) NOT LIKE '?%'
      GROUP BY  1, 2, 3, 4, 5, 6, 7, 8, 9
    ))
"""
predictions = client.query(query_prediction).to_dataframe()

Veremos unas cuantas predicciones y el resultado esperado:

In [ ]:
predictions[['income_bracket', 'predicted_income_bracket']].head()

In [ ]:
predictions['predicted_income_bracket_probs'].head()

En este caso la primera probabilidad de predicción es ~ 0.20, que corresponde a <50K.

#### Cuente el número de predicciones correctas:

In [ ]:
_count = predictions['income_bracket'].str.strip().str.lower() == \
  predictions['predicted_income_bracket'].str.strip().str.lower()
# Predicciones grupales:
_count.value_counts(normalize=True) 
# Nota: Establezca normalize=False para ver resultados agrupados.